<h1 style="color: purple">Script to transfer Excel data into ElasticSearch</h1>

<h3 style="color: blue">Importing necessary libraries in our program</h3>
<p><em> In our program, we use two libraries, one is Pandas and other is elasticsearch
 </em>  </p>

In [33]:
from elasticsearch import Elasticsearch
import pandas as pd

<h3 style="color: blue">Converting excel file into Data-Frames</h3>
<p><em> As we can observe that our excel workbook have two separate sheets so we create two different dataframes for the purpose
 </em>  </p>

In [34]:
df1 = pd.read_excel("beginner_assignment01.xlsx", "group_listing")
df2 = pd.read_excel("beginner_assignment01.xlsx", "product_listing")

<h3 style="color: blue">Showing first 5 rows of our dataframe</h3>
<p><em>We check whether we have obtained and are reading the correct data-set</em></p>

In [35]:
df1.head()

,group name,group description,isActive
0,Group 1,Group Number 1,yes
1,Group 2,Group Number 2,yes
2,Group 3,Group Number 3,yes
3,Group 4,Group Number 4,yes
4,Group 5,Group Number 5,yes


In [36]:
df2.head()

,Product Name,Model Name,Product Serial No,Group Associated,product MRP (rs.)
0,Demo Product 1,Product Model 1,98765987659801,Group 1,1001
1,Demo Product 1,Product Model 1,98765987659802,Group 2,1002
2,Demo Product 1,Product Model 1,98765987659803,Group 3,1003
3,Demo Product 1,Product Model 1,98765987659804,Group 4,1004
4,Demo Product 1,Product Model 1,98765987659805,Group 5,1005


<h3 style="color: blue">Converting our data frame to list data structure</h3>
<p><em> This has been done so that it becomes easier to transfer the data given in the excel sheet to our Elasticsearch database which we will see in the upcoming codes
    </em>  </p>

In [37]:
group_list = df1.values.tolist()
product_list = df2.values.tolist()

<h3 style="color: blue">Connecting to elastic search</h3>
<p><em>We require the host which in this case is "101.53.136.181" and port number, which in this case is "9200"
    </em>  </p>

In [38]:
es = Elasticsearch(HOST="101.53.136.181", PORT=9200)

<h3 style="color: blue">Deleting the indices</h3>
<p><em> Here, we try and delete the indices with the notion that if the index is already present in the node of the cluster then, it may arise error
    </em>  </p>

In [75]:
es.indices.delete(index="grouplisting", ignore=400)
es.indices.delete(index="productlisting", ignore=400)

{'acknowledged': True}

<h3 style="color: blue">Creating the index "grouplisting"</h3>
<p><em>We create a new index named "grouplisting" 
    </em>  </p>

In [76]:
es.indices.create(index="grouplisting", ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'grouplisting'}

<h3 style="color: blue">Inserting data into the index "grouplisting"</h3>
<p><em>We insert into the index the required data from the sheet "group_listing" of the excel workbook
    </em>  </p>

In [77]:
count = 0
for i in group_list:
    data = {"group_name":i[0], "group_description":i[1], "isActive":i[2]}
    es.create(index="grouplisting", doc_type="groupList", body=data, id=count+1, refresh=True)
    count += 1

<h3 style="color: blue">Creating the index "productlisting"</h3>
<p><em>We create new index "productlisting"
    </em>  </p>

In [65]:
es.indices.create(index="productlisting", ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'productlisting'}

<h3 style="color: blue">Inserting data into the index "productlisting"</h3>
<p><em>We insert into the index the required data from the sheet "product_listing" of the excel workbook
    </em>  </p>

In [66]:
count = 0
for i in product_list:
    data = {"product_serial_no":i[2], "product_name":i[0], "model_name":i[1], "group_assc":i[3], "MRP":i[4]}
    es.create(index="productlisting", doc_type="productList", body=data, id=count+1, refresh=True)
    count += 1

<h3 style="color: blue">Getting the index data</h3>
<p><em>We use get() in-order to fetch the data embedded into the index which show us the index mapping
    </em>  </p>

In [78]:
es.get(index="grouplisting", doc_type="groupList", id=2)

{'_index': 'grouplisting',
 '_type': 'groupList',
 '_id': '2',
 '_version': 1,
 '_seq_no': 1,
 '_primary_term': 1,
 'found': True,
 '_source': {'group_name': 'Group 2',
  'group_description': 'Group Number 2',
  'isActive': 'yes'}}

In [68]:
es.get(index="productlisting", doc_type="productList", id=2)

{'_index': 'productlisting',
 '_type': 'productList',
 '_id': '2',
 '_version': 1,
 '_seq_no': 1,
 '_primary_term': 1,
 'found': True,
 '_source': {'product_serial_no': 98765987659802,
  'product_name': 'Demo Product 1',
  'model_name': 'Product Model 1',
  'group_assc': 'Group 2',
  'MRP': 1002}}